#### Dyskretna Transformacja Fouriera (DFT)

Na wejściu dostajemy wektor x, który ma n elementów. Na wyjściu oczekujemy wektora y, który również ma n elementów. Wektor y to wynik transformacji, wartości w tym wektorze są napewno zespolone. y = F_n * x

In [44]:
import math
import numpy as np
import time

Pierwszym krokiem jest obliczenie ksi (czynnik skracający), ze wzoru: ξ = e^(-2πi/n) = cos(2π/n) - i sin(2π/n)

In [29]:
def calculate_ksi(n):
    angle = 2 * np.pi / n
    real = np.cos(angle)
    imaginary = -np.sin(angle)
    ksi = complex(real, imaginary)
    return ksi

Drugim krokiem jest konstrukcja macierzy Fouriera F_n. Macierz ma wymiary n x n. Element macierzy w j-tym wierszu i k-tej kolumnie to [F_n]_jk = ξ^(j*k)

In [30]:
def create_matrix(n):
    ksi = calculate_ksi(n)
    matrix = np.zeros((n,n),dtype=complex)
    for k in range (0,n):
        for j in range (0,n):
            potega = j * k
            t = ksi ** potega
            matrix[j,k] = t
    return matrix

Trzecim krokiem jest mnożenie macierzy F_n przez wektor x (aby uzyskać y)

In [31]:
def DFT(x):
    n = x.shape[0]
    return create_matrix(n) @ x    

Test:

In [32]:
x = np.array([2, 3, 4, 5])
print(DFT(x))

[14.+0.0000000e+00j -2.+2.0000000e+00j -2.-1.2246468e-15j
 -2.-2.0000000e+00j]


In [33]:
r = np.fft.fft(x)
print(r)

[14.+0.j -2.+2.j -2.+0.j -2.-2.j]


####  Odwrotna Dyskretna Transformacja Fouriera (IDFT)

Cel: Zaimplementować funkcję, która oblicza IDFT wektora y (który jest wynikiem DFT) i zwraca oryginalny wektor x.

F_n^(-1) * y == sprzężenie_zespolone((F_n * sprzężenie_zespolone(y))) / n

In [34]:
def IDFT(y):
    n = y.shape[0]
    #sprzężenie:
    y_conj = np.conjugate(y)
    
    F_n = create_matrix(n)
    x = np.conjugate(y_conj @ F_n) /n
    return x

In [35]:
print(IDFT(r))

[2.-0.00000000e+00j 3.+1.14423775e-17j 4.-1.22464680e-16j
 5.-5.33927493e-16j]


In [36]:
print(np.fft.ifft(r))

[2.+0.j 3.+0.j 4.+0.j 5.+0.j]


#### Algorytm Cooleya-Turkeya

Implementacja rekurencyjnego algorytmu Cooleya-Tukeya to klasyczne zadanie pokazujące moc algorytmów "dziel i zwyciężaj" w kontekście FFT.
Idea jest następująca:
DFT N-punktowego sygnału x można wyrazić za pomocą dwóch DFT N/2-punktowych.

Definicja Dyskretnej Transformacji Fouriera (DFT) dla $N$-punktowego sygnału $x_j$:
$$X_k = \sum_{j=0}^{N-1} x_j \cdot e^{-2\pi i \cdot jk / N} \quad \text{dla } k = 0, \ldots, N-1$$

Rozdzielamy sumę na wyrazy o indeksach parzystych ($j = 2m$) i nieparzystych ($j = 2m+1$). Zakładamy, że $N$ jest parzyste.
$$X_k = \sum_{m=0}^{N/2-1} x_{2m} \cdot e^{-2\pi i \cdot (2m)k / N} + \sum_{m=0}^{N/2-1} x_{2m+1} \cdot e^{-2\pi i \cdot (2m+1)k / N}$$

Upraszczamy wykładniki:
Dla wyrazów parzystych:
$$e^{-2\pi i \cdot (2m)k / N} = e^{-2\pi i \cdot mk / (N/2)}$$
Dla wyrazów nieparzystych:
$$e^{-2\pi i \cdot (2m+1)k / N} = e^{-2\pi i \cdot (2mk + k) / N} = e^{-2\pi i \cdot k / N} \cdot e^{-2\pi i \cdot 2mk / N} = e^{-2\pi i \cdot k / N} \cdot e^{-2\pi i \cdot mk / (N/2)}$$

Niech $W_N^k = e^{-2\pi i \cdot k / N}$ to tzw. "czynnik skręcający" (twiddle factor).
Zauważmy, że $W_N^{2k} = e^{-2\pi i \cdot 2k / N} = e^{-2\pi i \cdot k / (N/2)} = W_{N/2}^k$.

Podstawiając, otrzymujemy:
$$X_k = \sum_{m=0}^{N/2-1} x_{2m} \cdot W_{N/2}^{mk} + W_N^k \cdot \sum_{m=0}^{N/2-1} x_{2m+1} \cdot W_{N/2}^{mk}$$

Pierwsza suma to DFT $(N/2)$-punktowego sygnału złożonego z parzystych próbek $x$ (oznaczmy ją $E_k = \text{DFT}(x_{\text{even}})$).
Druga suma (bez $W_N^k$) to DFT $(N/2)$-punktowego sygnału złożonego z nieparzystych próbek $x$ (oznaczmy ją $O_k = \text{DFT}(x_{\text{odd}})$).

Zatem:
$$X_k = E_k + W_N^k \cdot O_k$$

Ta formuła oblicza pierwsze $N/2$ wartości $X_k$ (dla $k = 0, \ldots, N/2 - 1$).
Dla pozostałych $N/2$ wartości (tzn. dla $k' = k + N/2$, gdzie $k = 0, \ldots, N/2 - 1$), wykorzystujemy właściwości:
1.  Okresowości $E_k$ i $O_k$: $E_{k+N/2} = E_k$ oraz $O_{k+N/2} = O_k$ (ponieważ $E_k$ i $O_k$ są transformatami $(N/2)$-punktowymi).
2.  Właściwość czynnika skręcającego: $W_N^{k+N/2} = W_N^k \cdot W_N^{N/2} = W_N^k \cdot e^{-2\pi i \cdot (N/2) / N} = W_N^k \cdot e^{-\pi i} = W_N^k \cdot (-1) = -W_N^k$.

Zatem dla $k' = k + N/2$:
$$X_{k+N/2} = E_{k+N/2} + W_N^{k+N/2} \cdot O_{k+N/2}$$
$$X_{k+N/2} = E_k - W_N^k \cdot O_k$$

Te dwie formuły są sercem rekurencji dla $k = 0, \ldots, N/2 - 1$:
1.  $X_k = E_k + W_N^k \cdot O_k$
2.  $X_{k+N/2} = E_k - W_N^k \cdot O_k$

In [37]:
def fourierDiag(n):
    e = complex(math.cos(2*math.pi / n), - math.sin(2 * math.pi / n))
    return np.array([np.power(e, i) for i in range(n//2)], dtype=complex)

In [38]:
def fft_recursive(x):
    n = x.shape[0]
    if n == 2:
        return DFT(x)
    else:
        x_even = x[::2]
        x_odd = x[1::2]
        d = fourierDiag(n)
        
        E = myFFT(x_even)
        O = myFFT(x_odd)
        return np.append(E + (d * O), E - (d * O))

In [43]:
fft_recursive(x)

array([14.+0.j, -2.+2.j, -2.+0.j, -2.-2.j])

In [45]:
if __name__ == '__main__':
    x_test1 = np.random.rand(1024)

    start_time_my = time.perf_counter()
    X_test1_my = fft_recursive(x_test1)
    end_time_my = time.perf_counter()
    czas_my_1 = end_time_my - start_time_my

    start_time_np = time.perf_counter()
    X_test1_np = np.fft.fft(x_test1)
    end_time_np = time.perf_counter()
    czas_np_1 = end_time_np - start_time_np
    print(f"Czas mojej FFT: {czas_my_1:.8f} s")
    print(f"Czas NumPy FFT: {czas_np_1:.8f} s")
    print(f"Czy wyniki są bliskie? {np.allclose(X_test1_my, X_test1_np)}")

Czas mojej FFT: 0.02252823 s
Czas NumPy FFT: 0.00004282 s
Czy wyniki są bliskie? True
